In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_step_size_23h.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29 00:00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-01-29 23:00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,2.0,0.0,1.0,2.0,0.0,0.0,2022-01-30 22:00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,1.0,2022-01-31 21:00:00,1,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-02-01 20:00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93275,12.0,11.0,19.0,31.0,30.0,14.0,6.0,4.0,4.0,7.0,...,0.0,0.0,2.0,6.0,2.0,1.0,1.0,2022-12-27 13:00:00,265,2.0
93276,8.0,16.0,22.0,26.0,21.0,39.0,20.0,4.0,5.0,7.0,...,0.0,0.0,1.0,0.0,0.0,3.0,3.0,2022-12-28 12:00:00,265,2.0
93277,10.0,8.0,4.0,15.0,23.0,27.0,19.0,13.0,7.0,6.0,...,2.0,1.0,1.0,1.0,0.0,2.0,5.0,2022-12-29 11:00:00,265,2.0
93278,9.0,15.0,9.0,17.0,16.0,24.0,19.0,22.0,9.0,8.0,...,1.0,0.0,4.0,1.0,1.0,0.0,0.0,2022-12-30 10:00:00,265,5.0


In [10]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 10, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(67840, 674)
y_train.shape=(67840,)
X_test.shape=(25440, 674)
y_test.shape=(25440,)


In [11]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.ndarray:
        return X_test['rides_previous_1_hour']

In [12]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0        0.0
1        3.0
2        0.0
3        1.0
4        0.0
        ... 
25435    1.0
25436    3.0
25437    5.0
25438    0.0
25439    0.0
Name: rides_previous_1_hour, Length: 25440, dtype: float32

In [14]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.1124


In [16]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t-7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.ndarray:
        return X_test[f'rides_previous_{7*24}_hour']

In [17]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [18]:
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.1509


In [19]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual average demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        """"""
        return (
            X_test[f'rides_previous_{7*24}_hour'] + \
            X_test[f'rides_previous_{2*7*24}_hour'] + \
            X_test[f'rides_previous_{3*7*24}_hour'] + \
            X_test[f'rides_previous_{4*7*24}_hour']
        ) * 0.25

In [20]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.9716


Now for the step_size=1 dataset (the most representative one)

In [21]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_step_size_1h.parquet')

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 10, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(1558200, 674)
y_train.shape=(1558200,)
X_test.shape=(584855, 674)
y_test.shape=(584855,)


In [22]:
model_baseline1 = BaselineModelPreviousHour()
predictions_baseline1 = model_baseline1.predict(X_test)

model_baseline2 = BaselineModelPreviousWeek()
predictions_baseline2 = model_baseline2.predict(X_test)

model_baseline3 = BaselineModelLast4Weeks()
predictions_baseline3 = model_baseline3.predict(X_test)

In [24]:
test_mae_baseline1 = mean_absolute_error(y_test, predictions_baseline1)
test_mae_baseline2 = mean_absolute_error(y_test, predictions_baseline2)
test_mae_baseline3 = mean_absolute_error(y_test, predictions_baseline3)

# create a dataframe with the results
df_results = pd.DataFrame({
    'model': ['Baseline1', 'Baseline2', 'Baseline3'],
    'test_mae_stepsize=1': [test_mae_baseline1, test_mae_baseline2, test_mae_baseline3],
    'test_mae_stepsize=23': [4.1124, 4.1509, 3.9716],
})
df_results

,model,test_mae_stepsize=1,test_mae_stepsize=23
0,Baseline1,4.189563,4.1124
1,Baseline2,4.264975,4.1509
2,Baseline3,3.984459,3.9716
